In [121]:
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader
import re
import faiss
import numpy as np

In [85]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [11]:
'''
texts = [
    "Medical leave policy allows 12 paid leaves per year.",
    "Employees can apply medical leave with a doctor's certificate.",
    "Refunds are processed within 5 business days.",
    "Thilipan is such a good boy"
]
'''

In [14]:
#embeding = model.encode(texts, normalize_embeddings=True)

In [19]:
#embeding.shape

(4, 384)

In [183]:
def pdf_to_join(path):
    page = PdfReader(path)
    page_aech = [p.extract_text() for p in page.pages]
    full_pdf = ('\n').join(page_each)
    return full_pdf

def chunk_embed(text):
    text = re.sub(r'\n+','\n', text)
    patter_regex = r"\(\s*[ivxlcdm]+\s*\)"
    chunk = re.split(patter_regex, text, flags = re.IGNORECASE)

    chunks = []
    for i in chunk:
        ch = i.strip()
        if len(ch)>50:
            chunks.append(ch)

    return chunks

In [182]:
page = PdfReader("C:\\Users\\Thilipan\\Downloads\\Student-Code-of-Conduct.pdf")
page_each = [p.extract_text() for p in page.pages]

In [34]:
len(page_each)

10

In [37]:
full_pdf = ('\n').join(page_each)

In [81]:
def chunk_embed(text):
    text = re.sub(r'\n+','\n', text)
    patter_regex = r"\(\s*[ivxlcdm]+\s*\)"
    chunk = re.split(patter_regex, text, flags = re.IGNORECASE)

    chunks = []
    for i in chunk:
        ch = i.strip()
        if len(ch)>50:
            chunks.append(ch)

    return chunks

chunks = chunk_embed(full_pdf)
len_sing = [len(i) for i in chunks]
summ =  sum(len_sing)
leng = len(chunks)
mean = summ/leng
mean

313.1095890410959

In [69]:
text = re.sub(r'\n+','\n', full_pdf)
patter_regex = r"\(\s*[ivxlcdm]+\s*\)"
chunk = re.split(patter_regex, text, flags = re.IGNORECASE)
'''
for i in range(int(len(chunk)/2)):
    print(len(chunk[i]))
'''

'\nfor i in range(int(len(chunk)/2)):\n    print(len(chunk[i]))\n'

In [ ]:
chunks[50]

In [102]:
document = []

for i, chunk in enumerate(chunks):
    document.append({
        "id" : f"code_of_conduct_chunk{i}",
        "text" : chunk,
        "doc" : "Student-Code-of-Conduct.pdf"
    })

In [103]:
document[2]

{'id': 'code_of_conduct_chunk2',
 'text': 'Participation in an illegal or unjustified strike or abetting, inciting, instigating or acting in furtherance thereof, whether alone or in combination with others.',
 'doc': 'Student-Code-of-Conduct.pdf'}

In [109]:
text = [i['text'] for i in document]
len(text)

73

In [110]:
embeddings = model.encode(text, normalize_embeddings=True)

In [113]:
embeddings[3][:30]

array([-0.07442372,  0.06163823,  0.04807689, -0.03709005, -0.00116478,
        0.02743151,  0.05689528, -0.06960794,  0.0057109 ,  0.03528752,
        0.0755114 ,  0.02045024,  0.02325893, -0.03649571, -0.01273585,
       -0.08102009, -0.02627411,  0.03759187,  0.01421586,  0.02386038,
        0.03431748, -0.0433992 , -0.0427881 , -0.00778294,  0.00463974,
        0.02382512,  0.03119604,  0.02140358, -0.0465395 , -0.05417428],
      dtype=float32)

In [115]:
embeddings.dtype

dtype('float32')

In [119]:
embeddings.shape

(73, 384)

In [125]:
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

In [126]:
index.ntotal

73

In [197]:
query = "what happens if i smoke"
query_embed = model.encode(query, normalize_embeddings=True)
query_embed.shape

(384,)

In [198]:
query_array = np.array([query_embed])
query_array.shape

(1, 384)

In [199]:
score, indice = index.search(query_array, k=5)
score, indice

(array([[0.47370756, 0.32808018, 0.27143988, 0.21027778, 0.2051278 ]],
       dtype=float32),
 array([[68, 42, 41, 35,  6]]))

In [200]:
text[68]

'Smoking within the institute or possession of tobacco in any form or manner or e-cigarettes.'

In [153]:
sum(score[0].tolist())

0.8779017478227615